## Практическое задание 6. Градиентный бустинг ~~своими руками~~

Поздравляю! Это финальное упражнение в нашем курсе. Проявите все своё старание, терпение и опыт, чтобы выполнить его.  
Теперь вы многое знаете из машинного обучения и для вас не составит сложности попробовать разные алгоритмы, новые библиотеки и применить их к реальной задаче.

__Задание 1. (0.5 балла)__

Мы будем использовать данные соревнования [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data). 
* Загрузите таблицу **application_train.csv**;
* Запишите в Y столбец с целевой переменной;
* Удалите ненужные столбцы (для этого воспользуйтесь описанием);
* Определите тип столбцов и заполните пропуски - стратегия произвольная;
* Разбейте выборку в соотношении 70:30 с random_state=0.

Так как в данных значительный дисбаланс классов, в качестве метрики качества везде будем использовать площадь под precision-recall кривой.

In [6]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [7]:
X0 = pd.read_csv('application_train.csv')
X0

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
for i in X0:
    if X0[i].dtype == object: X0[i] = X0[i].fillna('')
    elif X0[i].dtype == int: X0[i] = X0[i].fillna(0)
    elif X0[i].dtype == float: X0[i] = X0[i].fillna(0.)

In [9]:
Y = X0['TARGET'].values
X = X0.drop(['TARGET', 'SK_ID_CURR'], axis=1)
X_num = X.loc[:, X.dtypes != object]
num_columns = X_num.columns

In [10]:
x_train, x_test, y_train, y_test = train_test_split(np.array(X_num), Y, test_size=0.3, random_state=0)

__Задание 2. (1.5 балла)__

Обучите реализации градиентного бустинга LightGBM и Catboost на вещественных признаках без подбора параметров. 
Почему получилась заметная разница в качестве? 

В этом и последующих экспериментах необходимо измерять время обучения моделей.

In [2]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import average_precision_score
from time import time

In [19]:
start_time = time()
lgbm_model = LGBMClassifier()
lgbm_model.fit(x_train, y_train)
y_lgbm = lgbm_model.predict_proba(x_test)[:, 1]
print('lgbm classifier score:', average_precision_score(y_test, y_lgbm))
print("--- %s seconds ---" % (time() - start_time))

[LightGBM] [Info] Number of positive: 17485, number of negative: 197772
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11115
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 99
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081228 -> initscore=-2.425771
[LightGBM] [Info] Start training from score -2.425771
lgbm classifier score: 0.23159230793933594
--- 3.0795202255249023 seconds ---


In [21]:
start_time = time()
cat_model = CatBoostClassifier(task_type='GPU', loss_function='Logloss')
cat_model.fit(x_train, y_train, verbose=False)
y_cat = cat_model.predict_proba(x_test)[:, 1]
print('catboost classifier score:', average_precision_score(y_test, y_cat))
print("--- %s seconds ---" % (time() - start_time))

catboost classifier score: 0.23370653771009461
--- 28.339354515075684 seconds ---


**ВЫВОД**: LightGBM работает примерно в 9 раз быстрее Catboost, несмотря на практически одинаковый результат 

__Задание 3. (2 балла)__

Подберите с CV=3 оптимальные параметры алгоритмов, изменяя:

* глубину деревьев;
* количество деревьев;
* темп обучения;
* оптимизируемый функционал.

Проанализируйте соотношения глубины и количества деревьев в зависимости от алгоритма.

In [4]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
from sklearn.model_selection import GridSearchCV

CatBoost

In [23]:
params = {
    'learning_rate': [0.01, 0.04, 0.08]
}

cat_model = CatBoostClassifier(task_type='GPU', iterations=1000, verbose=False, 
                               loss_function='CrossEntropy', depth=6)
gs = GridSearchCV(cat_model, params, cv=3, scoring='average_precision', verbose=0)
gs.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000028C244EB650>,
             param_grid={'learning_rate': [0.01, 0.04, 0.08]},
             scoring='average_precision')

In [24]:
gs.best_params_

{'learning_rate': 0.04}

In [25]:
start = time()
cat_model = CatBoostClassifier(task_type='GPU', loss_function='CrossEntropy', depth=6, 
                              iterations=1000, learning_rate=0.04)
cat_model.fit(x_train, y_train, verbose=False)
y_cat = cat_model.predict_proba(x_test)[:, 1]
end = time()
print('catboost classifier score:', average_precision_score(y_test, y_cat))
print('time:', end - start)

catboost classifier score: 0.23486288379378856
time: 29.157753705978394


LightGBM

In [16]:
params = {
    'learning_rate': [0.01, 0.04, 0.08]
}

lgbm_model = LGBMClassifier( n_estimators=1000, metric='binary_logloss', max_depth=5,force_row_wise=True)
gs = GridSearchCV(lgbm_model, params, cv=3, scoring='average_precision', verbose=0)
gs.fit(x_train, y_train)

[LightGBM] [Info] Number of positive: 11656, number of negative: 131848
[LightGBM] [Info] Total Bins 11169
[LightGBM] [Info] Number of data points in the train set: 143504, number of used features: 99
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081224 -> initscore=-2.425829
[LightGBM] [Info] Start training from score -2.425829
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

Exception ignored on calling ctypes callback function: <function _log_callback at 0x0000027678F2FCE0>
Traceback (most recent call last):
  File "C:\Users\Миша\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\basic.py", line 257, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

GridSearchCV(cv=3,
             estimator=LGBMClassifier(force_row_wise=True, max_depth=5,
                                      metric='binary_logloss',
                                      n_estimators=1000),
             param_grid={'learning_rate': [0.01, 0.04, 0.08]},
             scoring='average_precision')

In [13]:
gs.best_params_

{'learning_rate': 0.01}

In [14]:
start = time()
lgbm_model = LGBMClassifier(n_estimators=1000, metric='binary_logloss', max_depth=5, 
                            learning_rate=0.01)
lgbm_model.fit(x_train, y_train)
y_lgbm = lgbm_model.predict_proba(x_test)[:, 1]
end = time()
print('lgbm classifier score:', average_precision_score(y_test, y_lgbm))
print('time:', end - start)

[LightGBM] [Info] Number of positive: 17485, number of negative: 197772
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069526 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11115
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 99
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081228 -> initscore=-2.425771
[LightGBM] [Info] Start training from score -2.425771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

**ВЫВОД** Несмотря на какие-то предупреждения, LightGBM всё равно работает быстрее

__Задание 4. (3.5 балла)__

Добавьте категориальные признаки к вещественным следующими способами:

* как OHE признаки;
* как счетчики со сглаживанием.

При подсчете счетчиков запрещается использование циклов. 

На получившихся датасетах подберите параметры у каждого из алгоритмов. Как меняется время, необходимое для обучения модели в зависимости от способа кодирования? Сравните полученные результаты с встроенными методами обработки категориальных признаков. 

как OHE признаки

In [17]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
one_hot_X = pd.get_dummies(X, drop_first=True)

In [18]:
x_train_ohe, x_test_ohe = train_test_split(one_hot_X, test_size=0.3, random_state=0)

In [19]:
params = {
    'learning_rate': [0.01, 0.04, 0.08]
}

cat_model = CatBoostClassifier(task_type='GPU', iterations=1000, verbose=False, 
                              depth=8, loss_function='CrossEntropy')
gs = GridSearchCV(cat_model, params, cv=3, scoring='average_precision', verbose=0)
gs.fit(x_train_ohe, y_train)

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x000002767981B110>,
             param_grid={'learning_rate': [0.01, 0.04, 0.08]},
             scoring='average_precision')

In [20]:
gs.best_params_

{'learning_rate': 0.01}

In [21]:
start = time()
cat_model = CatBoostClassifier(task_type='GPU', loss_function='CrossEntropy', depth=8, 
                              iterations=1000, learning_rate=0.04)
cat_model.fit(x_train_ohe, y_train, verbose=False)
y_cat = cat_model.predict_proba(x_test_ohe)[:, 1]
end = time()
print('catboost classifier score:', average_precision_score(y_test, y_cat))
print('time:', end - start)

catboost classifier score: 0.2426371099856115
time: 23.16281294822693


**Score и время улучшились**

LightGBM

In [22]:
params = {
    'learning_rate': [0.01, 0.04, 0.08]
}

lgbm_model = LGBMClassifier(n_estimators=1000, max_depth=6, metric='binary_logloss')
gs = GridSearchCV(lgbm_model, params, cv=3, scoring='average_precision', verbose=0)
gs.fit(x_train_ohe, y_train)

ValueError: 
All the 9 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Миша\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Миша\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\sklearn.py", line 1284, in fit
    super().fit(
  File "C:\Users\Миша\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\sklearn.py", line 955, in fit
    self._Booster = train(
                    ^^^^^^
  File "C:\Users\Миша\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\engine.py", line 282, in train
    booster = Booste

In [23]:
gs.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [24]:
start = time()
lgbm_model = LGBMClassifier(n_estimators=1000, max_depth=6, metric='binary_logloss',
                           learning_rate=0.01)
lgbm_model.fit(x_train_ohe, y_train)
y_lgbm = lgbm_model.predict_proba(x_test_ohe)[:, 1]
end = time()
print('lgbm classifier score:', average_precision_score(y_test, y_lgbm))
print('time:', end - start)

LightGBMError: Do not support special JSON characters in feature name.

#### __Задание 5. (1 балл)__

Реализуйте блендинг подобранных в предыдущем задании моделей и сравните качество.

__Задание 6. (1.5 балла)__

В задании 3 вы подобрали гиперпараметры для LightGBM и CatBoost на вещественных признаках. Визуализируйте важности признаков, посчитанные этими алгоритмами, в виде горизонтального bar-plot (отсортируйте признаки по убыванию важности, подпишите названия признаков по оси y).

Для каждого из двух алгоритмов удалите неважные признаки (обычно по bar-plot хорошо видно порог на важность, с которого начинается хвост неважных признаков) и обучите ту же модель на получившихся данных. Сильно ли упало качество при удалении признаков, которые модель считает неважными?